# Import libraries

In [1]:
import cnn_models
import data_preparation
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from sklearn.svm import SVC
import tensorflow as tf
import data_augmentation
import random

/Users/kappa/opt/miniconda3/envs/dsim/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/kappa/opt/miniconda3/envs/dsim/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/kappa/opt/miniconda3/envs/dsim/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/kappa/opt/miniconda3/envs/dsim

# Fix seed

In [2]:
SEED = 10
random.seed(SEED)
tf.random.set_random_seed(SEED)

# Load recordings
## STANDARD RECORDINGS - No spectrogram normalization

In [3]:
recordings = data_preparation.load_recordings(paths=['recordings', 'output'])

Loading from recordings



Loading from output


Raw recordings have different lengths? Let's check it out:

In [4]:
min_y = min(map(np.shape, recordings))[0]
max_y = max(map(np.shape, recordings))[0]
print(min_y, max_y)

2784 50335


Yes! They vary a lot. For this reason we can add 0s at the beginning and at the end in order to uniform them

**TO DO: Another strategy may be to vary spectrogram params so that spectograms will have the same length**

In [5]:
pad_recordings = data_preparation.pad_zeros(recordings)

pad_zeros >>>
pad_zeros <<<


What is the range now?

In [6]:
min_y = min(map(np.shape, pad_recordings))[0]
max_y = max(map(np.shape, pad_recordings))[0]
print(min_y, max_y)

50335 50335


We can now compute spectograms:

In [7]:
spects = [data_preparation.compute_spectrogram(x) for x in pad_recordings]
spects = np.array(spects)

The procedure worked as expected! we can now move on to the prediction task

In [8]:
labels_speakers = data_preparation.load_labels(paths=['recordings', 'output'], label_type="speakers")
labels_digits = data_preparation.load_labels(paths=['recordings', 'output'])

Normalize spectrograms

In [9]:
norm_spects = [data_preparation.compute_spectrogram(x, normalize=True) for x in pad_recordings]
norm_spects = np.array(norm_spects)

## Augmentation

In [10]:
%%time
X_train_digit, y_train_digit, X_test_digit, y_test_digit = data_preparation.prepare_augmented_recordings(audio_dirs= ['output', 'recordings'],
                             y_type= ['digit', 'digit'],
                             n_category_test=15,
                             include_pitch=True)

HERE!
conversion_done!
compute_spectrograms >>>
pad_zeros >>>
pad_zeros <<<
pad_zeros >>>
pad_zeros <<<
Padding done
compute_spectrograms <<<
CPU times: user 5min 23s, sys: 19.6 s, total: 5min 42s
Wall time: 4min 40s


In [11]:
print("Lengths : {}, {}, {}, {}".format(len(X_train_digit),
                                                 len(y_train_digit),
                                                 len(X_test_digit),
                                                 len(y_test_digit),))

Lengths : 22000, 22000, 300, 300


In [48]:
%%time
X_train_speaker, y_train_speaker, X_test_speaker, y_test_speaker = data_preparation.prepare_augmented_recordings(audio_dirs= ['output', 'recordings'],
                             y_type= ['speakers_us', 'speakers_default'],
                             n_category_test=15,
                             include_pitch=True)

HERE!
conversion_done!
compute_spectrograms >>>
pad_zeros >>>
pad_zeros <<<
pad_zeros >>>
pad_zeros <<<
Padding done
compute_spectrograms <<<
CPU times: user 5min 45s, sys: 22.4 s, total: 6min 7s
Wall time: 4min 42s


In [13]:
print("Lengths : {}, {}, {}, {}".format(len(X_train_speaker),
                                                 len(y_train_speaker),
                                                 len(X_test_speaker),
                                                 len(y_test_speaker)))

Lengths : 22000, 22000, 300, 300


# Standard recordings
## Numbers

Split data in train and test

In [15]:
X_train, X_test, y_train, y_test = data_preparation.split_train_test_baseline_spectrograms(spects, labels_digits)

In [16]:
clf1 = SVC(kernel='rbf', class_weight='balanced', gamma="auto")

In [17]:
%%time
clf1 = clf1.fit(X_train, y_train)

CPU times: user 4min 2s, sys: 4.49 s, total: 4min 6s
Wall time: 5min 1s


In [18]:
%%time
y_pred = clf1.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.20      0.32        44
           1       0.18      0.78      0.29        45
           2       0.87      0.26      0.40        50
           3       0.39      0.58      0.47        45
           4       0.33      0.53      0.40        55
           5       0.77      0.40      0.52        43
           6       0.47      0.14      0.21        51
           7       0.83      0.32      0.46        47
           8       0.88      0.17      0.28        42
           9       0.79      0.39      0.53        38

    accuracy                           0.38       460
   macro avg       0.63      0.38      0.39       460
weighted avg       0.62      0.38      0.39       460

CPU times: user 38.8 s, sys: 767 ms, total: 39.5 s
Wall time: 46.6 s


### Normalize spectrograms

In [19]:
X_train, X_test, y_train, y_test = data_preparation.split_train_test_baseline_spectrograms(norm_spects, labels_digits)

In [20]:
%%time
clf1 = SVC(kernel='rbf', class_weight='balanced', gamma="auto")
clf1 = clf1.fit(X_train, y_train)

CPU times: user 2min 49s, sys: 3.02 s, total: 2min 52s
Wall time: 3min 16s


In [21]:
%%time
y_pred = clf1.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.89      0.88        44
           1       0.93      0.87      0.90        45
           2       0.57      0.94      0.71        50
           3       0.87      0.44      0.59        45
           4       1.00      0.89      0.94        55
           5       0.85      0.95      0.90        43
           6       0.56      0.69      0.62        51
           7       0.80      0.85      0.82        47
           8       0.76      0.60      0.67        42
           9       0.96      0.63      0.76        38

    accuracy                           0.78       460
   macro avg       0.82      0.77      0.78       460
weighted avg       0.81      0.78      0.78       460

CPU times: user 35.4 s, sys: 579 ms, total: 36 s
Wall time: 39 s


### CNNs

#### Normalized spectrograms

In [23]:
X_train, X_test, y_train, y_test, input_shape = data_preparation.split_train_test_nn(norm_spects, labels_digits)

In [24]:
model = cnn_models.paper_architecture(10, input_shape=input_shape)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 63, 156, 32)       544       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 77, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 37, 64)        32832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 17, 64)         0         
_________________________________________________________________
flatten (Flatten)            (None, 6528)              0         
_________________________________________________________________
dense (Dense)        

In [25]:
%%time
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=3)
model.fit(X_train, y_train,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_test, y_test))

Train on 1840 samples, validate on 460 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1840/1840 [==============================] - 23s 12ms/sample - loss: 2.2652 - acc: 0.1663 - val_loss: 2.1910 - val_acc: 0.4239
Epoch 2/10
1840/1840 [==============================] - 21s 11ms/sample - loss: 2.0984 - acc: 0.2636 - val_loss: 1.8936 - val_acc: 0.4000
Epoch 3/10
1840/1840 [==============================] - 22s 12ms/sample - loss: 1.8501 - acc: 0.3451 - val_loss: 1.6263 - val_acc: 0.4652
Epoch 4/10
1840/1840 [==============================] - 22s 12ms/sample - loss: 1.6482 - acc: 0.4212 - val_loss: 1.3823 - val_acc: 0.5413
Epoch 5/10
1840/1840 [==============================] - 21s 11ms/sample - loss: 1.4481 - acc: 0.4859 - val_loss: 1.1894 - val_acc: 0.6239
Epoch 6/10
1840/1840 [==============================] - 21s 11ms/sample - loss: 1.3256 - acc: 0.5332 - val_loss: 1.0873 - val_acc: 0.6348
Epoch 7/10
1840/1840 [==============================] - 21s 11ms/sample - los

In [26]:
Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test)
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92        44
           1       0.75      0.89      0.82        45
           2       0.88      0.72      0.79        50
           3       0.74      0.62      0.67        45
           4       0.98      0.73      0.83        55
           5       0.72      0.95      0.82        43
           6       0.67      0.69      0.68        51
           7       0.77      0.85      0.81        47
           8       0.61      0.79      0.69        42
           9       0.90      0.68      0.78        38

    accuracy                           0.78       460
   macro avg       0.79      0.78      0.78       460
weighted avg       0.80      0.78      0.78       460



#### Standard spectrogram

In [27]:
X_train, X_test, y_train, y_test, input_shape = data_preparation.split_train_test_nn(spects, labels_digits)

In [28]:
model = cnn_models.paper_architecture(10, input_shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 63, 156, 32)       544       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 77, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 37, 64)        32832     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 17, 64)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6528)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               652900    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
__________

In [29]:
%%time
model.fit(X_train, y_train,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_test, y_test))

Train on 1840 samples, validate on 460 samples
Epoch 1/10
1840/1840 [==============================] - 21s 11ms/sample - loss: 2.3005 - acc: 0.1560 - val_loss: 2.1092 - val_acc: 0.2870
Epoch 2/10
1840/1840 [==============================] - 20s 11ms/sample - loss: 2.1971 - acc: 0.2245 - val_loss: 2.0836 - val_acc: 0.3109
Epoch 3/10
1840/1840 [==============================] - 20s 11ms/sample - loss: 2.1195 - acc: 0.2750 - val_loss: 2.0177 - val_acc: 0.3261
Epoch 4/10
1840/1840 [==============================] - 20s 11ms/sample - loss: 2.0684 - acc: 0.2995 - val_loss: 1.9839 - val_acc: 0.2978
Epoch 5/10
1840/1840 [==============================] - 21s 11ms/sample - loss: 2.0373 - acc: 0.3038 - val_loss: 1.8618 - val_acc: 0.3587
Epoch 6/10
1840/1840 [==============================] - 24s 13ms/sample - loss: 1.9330 - acc: 0.3451 - val_loss: 1.7749 - val_acc: 0.4109
Epoch 7/10
1840/1840 [==============================] - 24s 13ms/sample - loss: 1.8738 - acc: 0.3652 - val_loss: 1.7688 - val

In [30]:
Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test)
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.30      0.42        44
           1       0.45      0.31      0.37        45
           2       1.00      0.02      0.04        50
           3       0.22      0.93      0.35        45
           4       0.83      0.35      0.49        55
           5       0.63      0.60      0.62        43
           6       0.94      0.29      0.45        51
           7       0.62      0.32      0.42        47
           8       0.47      0.21      0.30        42
           9       0.33      0.82      0.47        38

    accuracy                           0.40       460
   macro avg       0.62      0.42      0.39       460
weighted avg       0.64      0.40      0.39       460



From what we can see normalising spectrograms is the way to go. Let's use it by default

## Speakers
### SVD

In [31]:
X_train, X_test, y_train, y_test = data_preparation.split_train_test_baseline_spectrograms(norm_spects, labels_speakers)

In [32]:
%%time
clf1 = SVC(kernel='rbf', class_weight='balanced', gamma="auto")
clf1 = clf1.fit(X_train, y_train)

CPU times: user 1min 46s, sys: 1.7 s, total: 1min 47s
Wall time: 1min 52s


In [33]:
%%time
y_pred = clf1.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      alinda       0.86      1.00      0.93        19
        gian       0.92      1.00      0.96        23
     jackson       0.98      0.98      0.98        98
      khaled       0.91      0.97      0.94        30
     nicolas       1.00      0.96      0.98        94
        theo       0.80      0.84      0.82        98
    yweweler       0.84      0.78      0.81        98

    accuracy                           0.90       460
   macro avg       0.90      0.93      0.91       460
weighted avg       0.90      0.90      0.90       460

CPU times: user 31.4 s, sys: 495 ms, total: 31.9 s
Wall time: 34.3 s


### CNN

For neural networks it is not possible to pass the labels as-is: we need to transform them in numbers. The safest way is through one-hot encoding

In [34]:
y, target_names = data_preparation.transform_categorical_y(labels_speakers)

In [35]:
X_train, X_test, y_train, y_test, input_shape = data_preparation.split_train_test_nn(norm_spects, y, number_mode=False)

In [36]:
model = cnn_models.paper_architecture(7, input_shape=input_shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 63, 156, 32)       544       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 30, 77, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 37, 64)        32832     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 17, 64)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6528)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 70)                457030    
_________________________________________________________________
dropout_2 (Dropout)          (None, 70)                0         
__________

In [37]:
%%time
model.fit(X_train, y_train,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_test, y_test))

Train on 1840 samples, validate on 460 samples
Epoch 1/10
1840/1840 [==============================] - 21s 11ms/sample - loss: 1.8496 - acc: 0.2299 - val_loss: 1.7506 - val_acc: 0.2630
Epoch 2/10
1840/1840 [==============================] - 20s 11ms/sample - loss: 1.6731 - acc: 0.3723 - val_loss: 1.5252 - val_acc: 0.5652
Epoch 3/10
1840/1840 [==============================] - 20s 11ms/sample - loss: 1.4443 - acc: 0.4973 - val_loss: 1.3066 - val_acc: 0.6239
Epoch 4/10
1840/1840 [==============================] - 20s 11ms/sample - loss: 1.2248 - acc: 0.5788 - val_loss: 1.2711 - val_acc: 0.5609
Epoch 5/10
1840/1840 [==============================] - 20s 11ms/sample - loss: 1.0909 - acc: 0.6228 - val_loss: 1.0178 - val_acc: 0.6761
Epoch 6/10
1840/1840 [==============================] - 20s 11ms/sample - loss: 0.9615 - acc: 0.6685 - val_loss: 0.8704 - val_acc: 0.7065
Epoch 7/10
1840/1840 [==============================] - 20s 11ms/sample - loss: 0.8555 - acc: 0.6967 - val_loss: 0.7604 - val

In [38]:
Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test)
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.42      0.59        19
           1       0.67      0.52      0.59        23
           2       0.77      0.94      0.84        98
           3       0.92      0.37      0.52        30
           4       0.94      0.96      0.95        94
           5       0.80      0.94      0.86        98
           6       0.91      0.85      0.88        98

    accuracy                           0.84       460
   macro avg       0.86      0.71      0.75       460
weighted avg       0.85      0.84      0.83       460



#### Paper - batch_normalisation=True

In [39]:
model = cnn_models.paper_architecture(7, input_shape, batch_normalisation=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 63, 156, 32)       544       
_________________________________________________________________
batch_normalization_v1 (Batc (None, 63, 156, 32)       128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 30, 77, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 37, 64)        32832     
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 14, 37, 64)        256       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 6, 17, 64)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 6528)              0         
__________

In [40]:
%%time
model.fit(X_train, y_train,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_test, y_test))

Train on 1840 samples, validate on 460 samples
Epoch 1/10
1840/1840 [==============================] - 52s 29ms/sample - loss: 1.1255 - acc: 0.6353 - val_loss: 1.9349 - val_acc: 0.2130
Epoch 2/10
1840/1840 [==============================] - 50s 27ms/sample - loss: 0.5760 - acc: 0.8342 - val_loss: 2.0859 - val_acc: 0.2130
Epoch 3/10
1840/1840 [==============================] - 47s 26ms/sample - loss: 0.3942 - acc: 0.9038 - val_loss: 2.1561 - val_acc: 0.2478
Epoch 4/10
1840/1840 [==============================] - 50s 27ms/sample - loss: 0.3590 - acc: 0.9114 - val_loss: 1.9939 - val_acc: 0.2196
CPU times: user 7min 6s, sys: 1min 19s, total: 8min 26s
Wall time: 3min 21s


In [41]:
Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test)
print(classification_report(Y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

      alinda       0.00      0.00      0.00        19
        gian       0.00      0.00      0.00        23
     jackson       0.00      0.00      0.00        98
      khaled       0.00      0.00      0.00        30
     nicolas       0.00      0.00      0.00        94
        theo       0.00      0.00      0.00        98
    yweweler       0.21      1.00      0.35        98

    accuracy                           0.21       460
   macro avg       0.03      0.14      0.05       460
weighted avg       0.05      0.21      0.07       460



/Users/kappa/opt/miniconda3/envs/dsim/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Data augmentation
## Speaker

In [49]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y_train_speaker = enc.fit_transform(np.array(y_train_speaker).reshape(-1, 1)).toarray()
y_test_speaker = enc.transform(np.array(y_test_speaker).reshape(-1, 1)).toarray()
label_0 = enc.inverse_transform(np.array([0, 0, 0, 0, 0, 0, 0]).reshape(1, -1))[0][0]
label_1 = enc.inverse_transform(np.array([0, 1, 0, 0, 0, 0, 0]).reshape(1, -1))[0][0]
label_2 = enc.inverse_transform(np.array([0, 0, 1, 0, 0, 0, 0]).reshape(1, -1))[0][0]
label_3 = enc.inverse_transform(np.array([0, 0, 0, 1, 0, 0, 0]).reshape(1, -1))[0][0]
label_4 = enc.inverse_transform(np.array([0, 0, 0, 0, 1, 0, 0]).reshape(1, -1))[0][0]
label_5 = enc.inverse_transform(np.array([0, 0, 0, 0, 0, 1, 0]).reshape(1, -1))[0][0]
label_6 = enc.inverse_transform(np.array([0, 0, 0, 0, 0, 0, 1]).reshape(1, -1))[0][0]
target_names = [label_0, label_1, label_2, label_3, label_4, label_5, label_6]

In [51]:
X_train_speaker = np.array(X_train_speaker)
X_test_speaker = np.array(X_test_speaker)

In [55]:
X_train_speaker = X_train_speaker.reshape(X_train_speaker.shape[0],
                                          X_train_speaker.shape[1],
                                          X_train_speaker.shape[2],
                                          1)
X_test_speaker = X_test_speaker.reshape(X_test_speaker.shape[0], X_test_speaker.shape[1], X_test_speaker.shape[2], 1)

In [56]:
input_shape = (X_train_speaker.shape[1], X_train_speaker.shape[2], 1)

In [57]:
model = cnn_models.paper_architecture(7, input_shape=input_shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 63, 56, 32)        544       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 30, 27, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 12, 64)        32832     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 6, 5, 64)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 70)                134470    
_________________________________________________________________
dropout_5 (Dropout)          (None, 70)                0         
__________

In [58]:
%%time
model.fit(X_train_speaker, y_train_speaker,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_test_speaker, y_test_speaker))

Train on 24145 samples, validate on 105 samples
Epoch 1/10
24145/24145 [==============================] - 99s 4ms/sample - loss: 1.3695 - acc: 0.4873 - val_loss: 1.2513 - val_acc: 0.5143
Epoch 2/10
24145/24145 [==============================] - 94s 4ms/sample - loss: 0.8986 - acc: 0.6644 - val_loss: 0.9401 - val_acc: 0.6190
Epoch 3/10
24145/24145 [==============================] - 96s 4ms/sample - loss: 0.6972 - acc: 0.7282 - val_loss: 0.6563 - val_acc: 0.6952
Epoch 4/10
24145/24145 [==============================] - 96s 4ms/sample - loss: 0.5868 - acc: 0.7718 - val_loss: 0.5878 - val_acc: 0.8000
Epoch 5/10
24145/24145 [==============================] - 97s 4ms/sample - loss: 0.5021 - acc: 0.7998 - val_loss: 0.7646 - val_acc: 0.7238
Epoch 6/10
24145/24145 [==============================] - 98s 4ms/sample - loss: 0.4313 - acc: 0.8291 - val_loss: 0.3210 - val_acc: 0.9238
Epoch 7/10
24145/24145 [==============================] - 99s 4ms/sample - loss: 0.3968 - acc: 0.8457 - val_loss: 0.34

In [59]:
Y_test = np.argmax(y_test_speaker, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test_speaker)
print(classification_report(Y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

      alinda       0.93      0.93      0.93        15
        gian       1.00      0.93      0.97        15
     jackson       0.88      1.00      0.94        15
      khaled       1.00      0.80      0.89        15
     nicolas       1.00      1.00      1.00        15
        theo       0.93      0.93      0.93        15
    yweweler       0.88      1.00      0.94        15

    accuracy                           0.94       105
   macro avg       0.95      0.94      0.94       105
weighted avg       0.95      0.94      0.94       105



### Batch_normalization = True

In [60]:
model = cnn_models.paper_architecture(7, input_shape=input_shape, batch_normalisation=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 63, 56, 32)        544       
_________________________________________________________________
batch_normalization_v1_4 (Ba (None, 63, 56, 32)        128       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 30, 27, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 14, 12, 64)        32832     
_________________________________________________________________
batch_normalization_v1_5 (Ba (None, 14, 12, 64)        256       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 6, 5, 64)          0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 1920)              0         
__________

In [61]:
%%time
model.fit(X_train_speaker, y_train_speaker,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_test_speaker, y_test_speaker))

Train on 24145 samples, validate on 105 samples
Epoch 1/10
24145/24145 [==============================] - 234s 10ms/sample - loss: 0.8579 - acc: 0.6865 - val_loss: 0.4487 - val_acc: 0.8857
Epoch 2/10
24145/24145 [==============================] - 215s 9ms/sample - loss: 0.4962 - acc: 0.8055 - val_loss: 0.1943 - val_acc: 0.9619
Epoch 3/10
24145/24145 [==============================] - 239s 10ms/sample - loss: 0.4143 - acc: 0.8406 - val_loss: 0.2986 - val_acc: 0.9143
Epoch 4/10
24145/24145 [==============================] - 271s 11ms/sample - loss: 0.3509 - acc: 0.8651 - val_loss: 0.1840 - val_acc: 0.9619
Epoch 5/10
24145/24145 [==============================] - 224s 9ms/sample - loss: 0.3128 - acc: 0.8809 - val_loss: 0.1060 - val_acc: 0.9619
Epoch 6/10
24145/24145 [==============================] - 212s 9ms/sample - loss: 0.2916 - acc: 0.8884 - val_loss: 0.0940 - val_acc: 0.9714
Epoch 7/10
24145/24145 [==============================] - 208s 9ms/sample - loss: 0.2587 - acc: 0.9025 - val_

In [62]:
Y_test = np.argmax(y_test_speaker, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test_speaker)
print(classification_report(Y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

      alinda       0.75      1.00      0.86        15
        gian       1.00      0.60      0.75        15
     jackson       1.00      0.80      0.89        15
      khaled       0.94      1.00      0.97        15
     nicolas       0.88      1.00      0.94        15
        theo       1.00      1.00      1.00        15
    yweweler       0.94      1.00      0.97        15

    accuracy                           0.91       105
   macro avg       0.93      0.91      0.91       105
weighted avg       0.93      0.91      0.91       105



### Different architecture
Let's change a bit the architecture and see if we can improve scores:

In [63]:
model = cnn_models.custom_cnn(7, input_shape=input_shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 63, 56, 32)        544       
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 30, 27, 64)        32832     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 14, 12, 64)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 10752)             0         
_________________________________________________________________
dense_21 (Dense)             (None, 128)               1376384   
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 7)                 903       
Total para

In [64]:
%%time
model.fit(X_train_speaker, y_train_speaker,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_test_speaker, y_test_speaker))

Train on 24145 samples, validate on 105 samples
Epoch 1/10
24145/24145 [==============================] - 175s 7ms/sample - loss: 0.9655 - acc: 0.6378 - val_loss: 0.6175 - val_acc: 0.8095
Epoch 2/10
24145/24145 [==============================] - 176s 7ms/sample - loss: 0.5075 - acc: 0.8052 - val_loss: 0.3901 - val_acc: 0.9048
Epoch 3/10
24145/24145 [==============================] - 175s 7ms/sample - loss: 0.3322 - acc: 0.8709 - val_loss: 0.4505 - val_acc: 0.8571
Epoch 4/10
24145/24145 [==============================] - 176s 7ms/sample - loss: 0.2645 - acc: 0.8980 - val_loss: 0.7727 - val_acc: 0.7810
Epoch 5/10
24145/24145 [==============================] - 180s 7ms/sample - loss: 0.2222 - acc: 0.9129 - val_loss: 0.1569 - val_acc: 0.9619
Epoch 6/10
24145/24145 [==============================] - 179s 7ms/sample - loss: 0.1794 - acc: 0.9304 - val_loss: 0.0522 - val_acc: 0.9810
Epoch 7/10
24145/24145 [==============================] - 177s 7ms/sample - loss: 0.1607 - acc: 0.9371 - val_los

In [65]:
Y_test = np.argmax(y_test_speaker, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test_speaker)
print(classification_report(Y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

      alinda       1.00      1.00      1.00        15
        gian       1.00      1.00      1.00        15
     jackson       0.93      0.93      0.93        15
      khaled       0.93      0.93      0.93        15
     nicolas       1.00      1.00      1.00        15
        theo       1.00      1.00      1.00        15
    yweweler       1.00      1.00      1.00        15

    accuracy                           0.98       105
   macro avg       0.98      0.98      0.98       105
weighted avg       0.98      0.98      0.98       105



# Test model 

In [ ]:
import sounddevice as sd
import subprocess

import time
import librosa

import IPython.display as ipd

import os
from scipy.io import wavfile as wav

In [ ]:
def pad_zeros_single_rec(rec, max_y):
    rec = np.array(rec)
    diff_in_y = max_y - rec.shape[0]
    if diff_in_y > 0:
        half_diff = int(diff_in_y/2)
        remaining_diff = diff_in_y-half_diff
        v = np.pad(rec, (half_diff, remaining_diff), 'constant', constant_values=0)
        return v
    else:
        return rec

In [ ]:
def create_recording(duration, rec_rate, name = "test.wav", output_dir = "test/"):
    print("Ready in 3...", end = "")
    time.sleep(1)
    print("2...", end = "")
    time.sleep(1)
    print("1...")
    time.sleep(1)
    print("Go.")
    rec = sd.rec(int(duration * rec_rate), samplerate=rec_rate, channels=1, blocking=True)
    print("Playing the recording.")
    sd.play(rec, rec_rate)

    # after hearing the recording, decide whether to record it again or continue to next number
    # if you type anything, record again
    # if you press enter, save current recording & go to next number
    ok = input("OK?")
    if ok == "":
        librosa.output.write_wav(output_dir+name, rec, rec_rate)
        return rec
    ipd.clear_output(wait=True)
    create_recording(duration, rec_rate)

In [ ]:
def trim_audio(file, input_dir="test/", output_dir="test/", db=-48):

    if not os.path.isdir(input_dir):
        print(f"There should be an input \"{input_dir}\" directory.")
        sys.exit(0)
    
    # create output directory if not there yet
    if not os.path.isdir(output_dir):
        os.makedirs(output_dir)
        
    temp1 = output_dir+"temp1.wav"
    temp2 = output_dir+"temp2.wav"
    temp3 = output_dir+"temp3.wav"
 
    subprocess.run(["ffmpeg", "-y", "-i", input_dir+file, "-af", f"silenceremove=1:0:{db}dB", temp1])
    subprocess.run(["ffmpeg", "-y", "-i", temp1, "-af", "areverse", temp2])
    subprocess.run(["ffmpeg", "-y", "-i", temp2, "-af", f"silenceremove=1:0.1:{db}dB", temp3])
    subprocess.run(["ffmpeg", "-y", "-i", temp3, "-af", "areverse", output_dir+file])
    
    os.remove(temp1)
    os.remove(temp2)
    os.remove(temp3)

In [ ]:
def test_NN(nn, max_y, target_names, answer = None, duration=2, rec_rate=8000, directory = "test/", filename = "test.wav"):
    create_recording(duration, rec_rate, filename, directory)   
    ipd.clear_output()
    trim_audio(filename, directory, directory)
    # _, rec = wav.read(directory + "/" + filename)
    rec, _ = librosa.core.load(directory + "/" + filename, sr = rec_rate)
    rec = pad_zeros_single_rec(rec, max_y)
    # sd.play(rec, rec_rate)
    rec = data_preparation.compute_spectrogram(rec, normalize=True)
    rec = rec[np.newaxis,:,:,np.newaxis]
    preds = nn.predict_classes(rec)
    print("Model prediction: {}".format(target_names[preds[0]]))
    if answer is not None:
        print(f"Correct answer {answer}")
    return preds

In [ ]:
max_y = len(data_augm_pad_recordings[1])

In [ ]:
pred = test_NN(model, max_y, target_names, answer = "gian")

# TO DO:
- [x] Set random seed
- [x] Use only original recordings in test set of augmented scenario
- Use proper validation set (optional: also crossvalidation) for picking best models and params
- Data augmentation also for digit recognition